# 2000-2025

In [4]:
import pandas as pd, requests, time
from datetime import datetime

START = "2000-01-01"
END   = "2025-10-31"

KW_BBVA = '( "BBVA" OR "Banco Bilbao Vizcaya Argentaria" OR BBVA.MC ) (sourcelang:es OR sourcelang:en)'
KW_SAN  = '( "Banco Santander" OR "Santander Group" OR SAN.MC ) (sourcelang:es OR sourcelang:en) NOT "provincia de Santander" NOT "Santander, Cantabria" NOT "Santander (Colombia)"'

KEYWORDS = [KW_BBVA, KW_SAN]

def week_chunks(start_date: str, end_date: str, step_days=7):
    s = pd.to_datetime(start_date); e = pd.to_datetime(end_date)
    cur = s
    out = []
    while cur <= e:
        end = min(cur + pd.Timedelta(days=step_days-1), e)
        out.append((cur, end))
        cur = end + pd.Timedelta(days=1)
    return out

def gdelt_query_window(keyword: str, start_dt: pd.Timestamp, end_dt: pd.Timestamp, maxrecords=250, tries=3):
    base = "https://api.gdeltproject.org/api/v2/doc/doc"
    s = start_dt.strftime("%Y%m%d%H%M%S")
    e = (end_dt + pd.Timedelta(hours=23, minutes=59, seconds=59)).strftime("%Y%m%d%H%M%S")
    params = {
        "query": keyword,
        "mode": "ArtList",
        "format": "json",
        "maxrecords": str(maxrecords),
        "sort": "DateDesc",
        "startdatetime": s,
        "enddatetime": e
    }
    for attempt in range(tries):
        try:
            r = requests.get(base, params=params, timeout=30)
            if r.status_code == 200:
                j = r.json()
                return j.get("articles", []) or []
            time.sleep(2**attempt)
        except Exception:
            time.sleep(2**attempt)
    return []

def collect_news_gdelt(keywords, start, end, step_days=7, sleep_s=0.25):
    chunks = week_chunks(start, end, step_days=step_days)
    all_rows = []
    for kw in keywords:
        for a,b in chunks:
            rows = gdelt_query_window(kw, a, b)
            if rows:
                for r in rows:
                    r["_kw"] = "BBVA" if kw == KEYWORDS[0] else "Santander"
                    all_rows.append(r)
            # log mínimo por ventana
            print(f"{kw[:25]}... {a.date()}→{b.date()} · {len(rows)} artículos")
            time.sleep(sleep_s)
    return pd.DataFrame.from_records(all_rows)



In [ ]:
news_raw = collect_news_gdelt(KEYWORDS, START, END, step_days=7)
print("Shape:", news_raw.shape)
news_raw.head()

( "BBVA" OR "Banco Bilbao... 2000-01-01→2000-01-07 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-01-08→2000-01-14 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-01-15→2000-01-21 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-01-22→2000-01-28 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-01-29→2000-02-04 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-02-05→2000-02-11 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-02-12→2000-02-18 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-02-19→2000-02-25 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-02-26→2000-03-03 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-03-04→2000-03-10 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-03-11→2000-03-17 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-03-18→2000-03-24 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-03-25→2000-03-31 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-04-01→2000-04-07 · 0 artículos
( "BBVA" OR "Banco Bilbao... 2000-04-08→2000-04-14 · 0 artículos
( "BBVA" OR "Banco Bilbao

In [ ]:
def normalize_gdelt(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df.copy()

    df = df.copy()
    # fecha
    if "seendate" in df.columns:
        def parse_see(x):
            try: return pd.to_datetime(x, format="%Y%m%d%H%M%S", errors="coerce")
            except: return pd.to_datetime(x, errors="coerce")
        df["dt"] = df["seendate"].apply(parse_see)
    elif "date" in df.columns:
        df["dt"] = pd.to_datetime(df["date"], errors="coerce")
    else:
        df["dt"] = pd.NaT

    # columnas mínimas
    for c in ["title","url","domain","sourcecountry","language"]:
        if c not in df.columns: df[c] = None

    out = df.rename(columns={"sourcecountry":"source_country"})[
        ["dt","language","source_country","domain","title","url","_kw"]
    ].dropna(subset=["dt"]).sort_values("dt").reset_index(drop=True)

    # dedup básicos
    out = out.drop_duplicates(subset=["url"])
    out = out.drop_duplicates(subset=["title","dt"])
    return out

news_norm = normalize_gdelt(news_raw)
print("Después de normalize:", news_norm.shape)
news_norm.head()


In [ ]:
test = gdelt_query_window(KEYWORDS[0], pd.to_datetime("2024-01-01"), pd.to_datetime("2024-01-07"))
print(len(test))
test[:1]


---

In [3]:
def daterange_chunks(start_date: str, end_date: str, freq="Q"):
    # genera [("2000-01-01","2000-03-31"), ("2000-04-01","2000-06-30"), ...]
    s = pd.to_datetime(start_date)
    e = pd.to_datetime(end_date)
    edges = pd.date_range(s, e, freq=freq).to_list()
    if not edges or edges[-1] < e:
        edges.append(e)
    # construir intervalos consecutivos
    chunks = []
    cur = s
    for edge in edges:
        if edge < cur: 
            continue
        chunk_end = min(edge, e)
        chunks.append((cur, chunk_end))
        cur = edge + pd.Timedelta(days=1)
        if cur > e:
            break
    return chunks

def gdelt_query(keyword: str, start_dt: pd.Timestamp, end_dt: pd.Timestamp, max_per_chunk=250):
    """
    Usa el endpoint Doc 2.1 (ArtList). Devuelve lista de artículos (dicts).
    GDELT limita 250 por request. Iteramos con sort=DateDesc y 'timespan' indirecto.
    """
    base = "https://api.gdeltproject.org/api/v2/doc/doc"
    # Formato timestamps: YYYYMMDDHHMMSS
    s = start_dt.strftime("%Y%m%d%H%M%S")
    e = (end_dt + pd.Timedelta(hours=23, minutes=59, seconds=59)).strftime("%Y%m%d%H%M%S")

    params = {
        "query": keyword,
        "mode": "ArtList",
        "format": "json",
        "maxrecords": str(max_per_chunk),
        "sort": "DateDesc",
        "startdatetime": s,
        "enddatetime": e
    }

    all_rows = []
    # Intentos con backoff por si hay throttling
    for attempt in range(3):
        try:
            r = requests.get(base, params=params, timeout=30)
            if r.status_code == 200:
                data = r.json()
                rows = data.get("articles", [])
                all_rows.extend(rows)
                break
            else:
                time.sleep(2**attempt)
        except Exception:
            time.sleep(2**attempt)
    return all_rows

def collect_news_gdelt(keywords, start, end, freq="Q", sleep_s=0.5):
    chunks = daterange_chunks(start, end, freq=freq)
    records = []
    for kw in keywords:
        for (a,b) in tqdm(chunks, desc=f"Descargando: {kw}"):
            rows = gdelt_query(kw, a, b)
            for r in rows:
                r["_kw"] = kw
                records.append(r)
            time.sleep(sleep_s)  # cortesía
    return pd.DataFrame.from_records(records)

news_raw = collect_news_gdelt(KEYWORDS, START, END, freq="Q")
print(news_raw.shape)
news_raw.head()


C:\Users\lopec\AppData\Local\Temp\ipykernel_41636\2901136136.py:5: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  edges = pd.date_range(s, e, freq=freq).to_list()
Descargando: "BBVA" OR "Banco Bilbao Vizcaya Argentaria" OR BBVA.MC: 100%|██████████| 104/104 [19:14<00:00, 11.10s/it]
Descargando: "Banco Santander" OR "Santander" OR SAN.MC: 100%|██████████| 104/104 [19:30<00:00, 11.25s/it]

(0, 0)


""


In [ ]:
raw_path = RAW_NEWS_DIR / "gdelt_bbva_santander_2000_2025.parquet"
news_raw.to_parquet(raw_path, index=False)
raw_path

In [ ]:
def normalize_gdelt(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df

    # columnas típicas: date, url, title, language, sourceCountry, domain, etc.
    df = df.copy()

    # fecha a datetime (GDELT da epochms o string dependiendo del caso)
    if "seendate" in df.columns:
        # algunos endpoints devuelven "seendate" como "YYYYMMDDHHMMSS"
        def parse_see(x):
            try:
                return pd.to_datetime(x, format="%Y%m%d%H%M%S", errors="coerce")
            except Exception:
                return pd.to_datetime(x, errors="coerce")
        df["dt"] = df["seendate"].apply(parse_see)
    elif "date" in df.columns:
        df["dt"] = pd.to_datetime(df["date"], errors="coerce")
    else:
        df["dt"] = pd.NaT

    # filtrar idiomas
    lang_col = "language" if "language" in df.columns else None
    if lang_col:
        df["language"] = df[lang_col].str.lower().fillna("")
        df = df[df["language"].isin(["spanish", "english"])]

    # renombrar campos
    rename_map = {
        "title": "title",
        "url": "url",
        "domain": "domain",
        "sourcecountry": "source_country",
    }
    # asegurar columnas
    for c in ["title","url","domain","sourcecountry"]:
        if c not in df.columns:
            df[c] = None

    out = df.rename(columns=rename_map)[["dt","language","source_country","domain","title","url","_kw"]].copy()
    out = out.dropna(subset=["dt"]).sort_values("dt").reset_index(drop=True)

    # deduplicados básicos
    out = out.drop_duplicates(subset=["url"]).drop_duplicates(subset=["title","dt"])
    return out

news_norm = normalize_gdelt(news_raw)
news_norm.shape, news_norm.head()


In [ ]:
norm_path = INT_NEWS_DIR / "news_norm_2000_2025.parquet"
news_norm.to_parquet(norm_path, index=False)
norm_path

In [ ]:
import re
from urllib.parse import urlparse

def simple_fetch_text(url, timeout=12):
    try:
        r = requests.get(url, timeout=timeout, headers={"User-Agent": "Mozilla/5.0"})
        if r.status_code!=200: 
            return None
        # extracción muy básica del <title> y párrafos
        html = r.text
        # cortar scripts/style
        html = re.sub(r"<script.*?</script>", " ", html, flags=re.S|re.I)
        html = re.sub(r"<style.*?</style>", " ", html, flags=re.S|re.I)
        # tags -> espacios
        text = re.sub(r"<[^>]+>", " ", html)
        # normalización básica
        text = re.sub(r"\s+", " ", text).strip()
        return text[:50000]  # limitar
    except:
        return None

# Pequeña muestra para probar:
sample = news_norm.head(50).copy()
sample["raw_text"] = sample["url"].apply(simple_fetch_text)
sample.head(2)


In [ ]:
# Sentimiento con pysentimiento (español)
from pysentimiento import create_analyzer

# Cargar una vez (puede tardar un poco en la primera)
analyzer = create_analyzer(task="sentiment", lang="es")

def infer_sentiment(text: str, lang: str):
    """
    Si lang es 'spanish' -> usamos analyzer ES
    Si lang es 'english' -> también probamos con ES (multilingüe suele aguantar)
    Mejoras: añadir segundo analyzer en inglés si vemos mucho contenido EN.
    """
    if not isinstance(text, str) or len(text.strip()) == 0:
        return None, None
    # Recortar para velocidad (los titulares + primeros 500-1000 chars suelen bastar)
    chunk = text[:1200]
    res = analyzer.predict(chunk)
    label = res.output  # POS/NEG/NEU
    score = res.probas.get(label, None)
    return label, score

# Para empezar, usaremos el TITLE como entrada (rápido).
news_for_sent = news_norm.copy()
news_for_sent["text_base"] = news_for_sent["title"].fillna("")

# Inferencia por lotes (titulares)
sent_labels, sent_scores = [], []
for t, lang in tqdm(zip(news_for_sent["text_base"], news_for_sent["language"]), total=len(news_for_sent)):
    lab, sco = infer_sentiment(t, lang)
    sent_labels.append(lab); sent_scores.append(sco)

news_for_sent["sent_label"] = sent_labels
news_for_sent["sent_score"] = sent_scores

news_for_sent.head()


In [ ]:
sent_titles_path = INT_NEWS_DIR / "news_sent_titles.parquet"
news_for_sent.to_parquet(sent_titles_path, index=False)
sent_titles_path

In [ ]:
df = news_for_sent.copy()
df["date"] = df["dt"].dt.date

# Mapear a score numérico sencillo
map_num = {"POS": 1, "NEU": 0, "NEG": -1, None: 0}
df["sent_num"] = df["sent_label"].map(map_num).fillna(0)

agg_daily = (df
             .groupby(["date","_kw"])
             .agg(n=("url","count"),
                  pos=("sent_label", lambda s: (s=="POS").sum()),
                  neg=("sent_label", lambda s: (s=="NEG").sum()),
                  neu=("sent_label", lambda s: (s=="NEU").sum()),
                  sent_mean=("sent_num","mean"))
             .reset_index())

# Pivot para tener BBVA y SAN en columnas
pivot_cols = ["n","pos","neg","neu","sent_mean"]
out_daily = (agg_daily
             .pivot(index="date", columns="_kw", values=pivot_cols)
             .sort_index())

# aplanar MultiIndex de columnas
out_daily.columns = [f"{m}__{kw}" for m,kw in out_daily.columns]
out_daily = out_daily.reset_index().rename(columns={"date":"Date"})

# Exportar
daily_path_csv = PROC_NEWS_DIR / "news_sentiment_daily.csv"
daily_path_parq = PROC_NEWS_DIR / "news_sentiment_daily.parquet"
out_daily.to_csv(daily_path_csv, index=False)
out_daily.to_parquet(daily_path_parq, index=False)

daily_path_csv, daily_path_parq


In [ ]:
df["week"] = pd.to_datetime(df["date"]) - pd.to_timedelta(pd.to_datetime(df["date"]).dt.weekday, unit="D")
df["month"] = pd.to_datetime(df["date"]).values.astype("datetime64[M]")

def agg_period(period_col):
    tmp = (df.groupby([period_col,"_kw"])
             .agg(n=("url","count"),
                  pos=("sent_label", lambda s: (s=="POS").sum()),
                  neg=("sent_label", lambda s: (s=="NEG").sum()),
                  neu=("sent_label", lambda s: (s=="NEU").sum()),
                  sent_mean=("sent_num","mean"))
             .reset_index())
    out = (tmp.pivot(index=period_col, columns="_kw", values=["n","pos","neg","neu","sent_mean"])
              .sort_index())
    out.columns = [f"{m}__{kw}" for m,kw in out.columns]
    out = out.reset_index().rename(columns={period_col:"Date"})
    return out

weekly = agg_period("week")
monthly = agg_period("month")

weekly.to_csv(PROC_NEWS_DIR / "news_sentiment_weekly.csv", index=False)
monthly.to_csv(PROC_NEWS_DIR / "news_sentiment_monthly.csv", index=False)

weekly.head(), monthly.head()


In [ ]:
import matplotlib.pyplot as plt

cols_bbva = [c for c in out_daily.columns if "__" in c and "BBVA" in c]
cols_san  = [c for c in out_daily.columns if "__" in c and "Santander" in c]

plt.figure(figsize=(12,5))
plt.plot(out_daily["Date"], out_daily["sent_mean__\"BBVA\" OR \"Banco Bilbao Vizcaya Argentaria\" OR BBVA.MC"], label="Sent diario · BBVA")
plt.plot(out_daily["Date"], out_daily["sent_mean__\"Banco Santander\" OR \"Santander\" OR SAN.MC"], label="Sent diario · SAN")
plt.title("Sentimiento medio diario (titulares) · 2000–2025")
plt.xlabel("Fecha"); plt.ylabel("sent_mean (-1..1)")
plt.legend(); plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
precios_bbva = pd.read_csv("../data/BBVA.csv", parse_dates=["Date"])
precios_san  = pd.read_csv("../data/SANTANDER.csv", parse_dates=["Date"])

sent_daily = pd.read_csv(PROC_NEWS_DIR / "news_sentiment_daily.csv", parse_dates=["Date"])

# Merge (alineamos por Date)
m_bbva = precios_bbva.merge(sent_daily[["Date","sent_mean__\"BBVA\" OR \"Banco Bilbao Vizcaya Argentaria\" OR BBVA.MC",
                                        "n__\"BBVA\" OR \"Banco Bilbao Vizcaya Argentaria\" OR BBVA.MC"]],
                            on="Date", how="left")

m_san  = precios_san.merge(sent_daily[["Date","sent_mean__\"Banco Santander\" OR \"Santander\" OR SAN.MC",
                                       "n__\"Banco Santander\" OR \"Santander\" OR SAN.MC"]],
                           on="Date", how="left")

m_bbva.head(), m_san.head()
